# Example of Verilog file processing

Configure the path to the Pyverilog directory

In [1]:
%config IPCompleter.greedy=True

In [2]:
import sys
pyverilog_path = "/work/Pyverilog"
sys.path.insert(0, pyverilog_path)

In [3]:
from pyverilog.vparser.parser import VerilogCodeParser
from pyverilog.dataflow.modulevisitor import ModuleVisitor
from pyverilog.dataflow.signalvisitor import SignalVisitor
from pyverilog.dataflow.bindvisitor import BindVisitor
from pyverilog.dataflow.optimizer import VerilogDataflowOptimizer
from pyverilog.dataflow.subset import VerilogSubset

In [4]:
verilog_file_path = "/work/Pyverilog/verilogcode/count.v"
%cat /work/Pyverilog/verilogcode/count.v

module TOP(CLK, RST_X);
  input CLK;
  input RST_X;
  reg [3:0] cnt;

  always @(posedge CLK or negedge RST_X) begin
    if(!RST_X) begin
      cnt <= 0;
    end else begin
      cnt <= cnt + 1;
    end
  end
  
endmodule


In [5]:
parser=VerilogCodeParser([verilog_file_path])
ast = parser.parse()

In [6]:
module_visitor = ModuleVisitor()
module_visitor.visit(ast)
moduleinfotable = module_visitor.get_moduleinfotable()

In [7]:
signal_visitor = SignalVisitor(moduleinfotable, "TOP")
signal_visitor.start_visit()
frametable = signal_visitor.getFrameTable()

In [8]:
bind_visitor = BindVisitor(moduleinfotable, "TOP", frametable, noreorder=False)
bind_visitor.start_visit()
dataflow = bind_visitor.getDataflows()

In [35]:
binddict=dataflow.getBinddict()
for key in binddict.keys():
    for b in binddict[key]:
        print(b.tostr())
        print(b.isCombination())
        print(b.getdest())

(Bind dest:TOP.cnt tree:(Branch Cond:(Operator Ulnot Next:(Terminal TOP.RST_X)) True:(IntConst 0) False:(Operator Plus Next:(Terminal TOP.cnt),(IntConst 1))))
False
TOP_cnt


In [36]:
optimizer = VerilogDataflowOptimizer(dataflow.getTerms(),  dataflow.getBinddict())
optimizer.resolveConstant()

In [37]:
resolved_terms = optimizer.getResolvedTerms()
resolved_binddict = optimizer.getResolvedBinddict()
constlist = optimizer.getConstlist()

In [38]:
for key in resolved_binddict.keys():
    for b in binddict[key]:
        print(b.tostr())
        print(b.isCombination())
        print(b.getdest())

(Bind dest:TOP.cnt tree:(Branch Cond:(Operator Ulnot Next:(Terminal TOP.RST_X)) True:(IntConst 0) False:(Operator Plus Next:(Terminal TOP.cnt),(IntConst 1))))
False
TOP_cnt
